# Intranet Analytics - Aggregated Data Analysis

This notebook uses **pre-aggregated data** for faster analysis and smaller dataset sizes.

**Available tables:**
- `fact_daily` - Daily metrics per page (UV, views, likes, comments)
- `fact_daily_website` - Daily metrics per website
- `fact_daily_employee` - Daily metrics by website + region + division (compact, for employee demographics)
- `fact_monthly` - Monthly metrics per page
- `dim_date` - Date dimension
- `page_inventory` - Page metadata
- `employee_contact` - Employee dimension (if employee_contact.csv provided)

**Note:** UV (Unique Visitors) is pre-calculated at the aggregation grain. For page-level employee analysis, use the detailed database.

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from pathlib import Path

# Suppress font warnings
warnings.filterwarnings('ignore', message='Glyph.*missing from font')

# Project paths
PROJECT_ROOT = Path.cwd().parent
DB_PATH = PROJECT_ROOT / "output" / "db" / "analytics_agg.duckdb"

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans', 'Helvetica', 'sans-serif']

# DataFrame display formatting
pd.set_option('display.float_format', lambda x: f'{x:,.0f}' if x == int(x) else f'{x:,.2f}')
pd.set_option('display.max_colwidth', 50)

# Integer columns for formatting
INTEGER_COLS = [
    'unique_visitors', 'views', 'visits', 'likes', 'comments', 'engagements',
    'row_count', 'pages_viewed', 'days_in_period'
]

def format_df(df):
    """Format dataframe with thousand separators and clean integers."""
    df_display = df.copy()
    for col in df_display.columns:
        if col in INTEGER_COLS or df_display[col].dtype in ['int64', 'Int64']:
            df_display[col] = df_display[col].apply(lambda x: f'{int(x):,}' if pd.notna(x) else '')
        elif df_display[col].dtype == 'float64':
            if df_display[col].dropna().apply(lambda x: x == int(x)).all():
                df_display[col] = df_display[col].apply(lambda x: f'{int(x):,}' if pd.notna(x) else '')
    return df_display

# Connect to the database
con = duckdb.connect(str(DB_PATH), read_only=True)
print(f"Connected to: {DB_PATH}")

---
## 1. Data Overview

In [ ]:
# Table overview
base_tables = ['fact_daily', 'fact_daily_website', 'fact_monthly', 'dim_date', 'page_inventory']
employee_tables = ['fact_daily_employee', 'employee_contact']

# Check which tables exist
all_db_tables = [t[0] for t in con.execute("SHOW TABLES").fetchall()]

print("=" * 50)
print("AGGREGATED DATABASE OVERVIEW")
print("=" * 50)

for table in base_tables:
    if table in all_db_tables:
        count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
        print(f"{table}: {count:,} rows")

# Check for employee tables
has_employee = 'fact_daily_employee' in all_db_tables
if has_employee:
    print("\n-- Employee Tables --")
    for table in employee_tables:
        if table in all_db_tables:
            count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
            print(f"{table}: {count:,} rows")

# Date range
date_range = con.execute("""
    SELECT MIN(date) as min_date, MAX(date) as max_date
    FROM fact_daily
""").fetchone()
print(f"\nDate range: {date_range[0]} to {date_range[1]}")
print("=" * 50)

---
## 2. Daily Website Metrics

In [ ]:
# Daily metrics by website
daily_website = con.execute("""
    SELECT
        date,
        day_name,
        is_weekend,
        websitename,
        unique_visitors,
        views,
        visits,
        likes,
        comments,
        engagements,
        pages_viewed
    FROM fact_daily_website
    ORDER BY date DESC, unique_visitors DESC
""").df()

print(f"Daily website metrics: {len(daily_website):,} rows")
format_df(daily_website.head(20))

---
## 3. Website Summary (All Time)

In [ ]:
# Total metrics by website
website_summary = con.execute("""
    SELECT
        websitename,
        SUM(unique_visitors) as total_uv_sum,
        SUM(views) as total_views,
        SUM(visits) as total_visits,
        SUM(likes) as total_likes,
        SUM(comments) as total_comments,
        SUM(engagements) as total_engagements,
        COUNT(DISTINCT date) as days_active
    FROM fact_daily_website
    GROUP BY websitename
    ORDER BY total_views DESC
""").df()

format_df(website_summary)

---
## 4. Monthly Trends

In [ ]:
# Monthly summary
monthly_summary = con.execute("""
    SELECT
        year_month,
        month_name,
        SUM(unique_visitors) as total_uv_sum,
        SUM(views) as total_views,
        SUM(visits) as total_visits,
        SUM(likes) as total_likes,
        SUM(comments) as total_comments,
        SUM(engagements) as total_engagements,
        COUNT(DISTINCT marketingpageid) as pages_active
    FROM fact_monthly
    GROUP BY year_month, month_name
    ORDER BY year_month
""").df()

print(f"Monthly summary:")
print(f"  Date range: {monthly_summary['year_month'].min()} to {monthly_summary['year_month'].max()}")
print()
format_df(monthly_summary)

---
## 5. Top Pages by Views (Monthly)

In [ ]:
# Top pages by total views
top_pages = con.execute("""
    SELECT
        pagename,
        websitename,
        theme,
        SUM(views) as total_views,
        SUM(unique_visitors) as total_uv_sum,
        SUM(likes) as total_likes,
        SUM(comments) as total_comments,
        SUM(engagements) as total_engagements
    FROM fact_monthly
    GROUP BY pagename, websitename, theme
    ORDER BY total_views DESC
    LIMIT 25
""").df()

format_df(top_pages)

---
## 6. Top Engaged Pages

In [ ]:
# Top pages by engagement
top_engaged = con.execute("""
    SELECT
        pagename,
        websitename,
        contenttype,
        SUM(likes) as total_likes,
        SUM(comments) as total_comments,
        SUM(engagements) as total_engagements,
        SUM(views) as total_views,
        ROUND(SUM(engagements) * 100.0 / NULLIF(SUM(views), 0), 2) as engagement_rate_pct
    FROM fact_monthly
    GROUP BY pagename, websitename, contenttype
    HAVING SUM(engagements) > 0
    ORDER BY total_engagements DESC
    LIMIT 25
""").df()

format_df(top_engaged)

---
## 7. Employee Analysis (if available)

Analysis by employee attributes from `fact_daily_employee` table.

**Grain:** Date + Website + Region + Division (compact aggregation)

For page-level employee analysis (e.g., UV by region for a specific URL), use the detailed database.

In [ ]:
# === EMPLOYEE ANALYSIS FILTERS ===
# Modify these to slice and dice employee metrics

# Website filter (case-insensitive contains, or None for all)
EMPLOYEE_FILTER_WEBSITE = None  # e.g., "intranet" or None for all

# Date range filter (YYYYMMDD format, or None for all)
EMPLOYEE_FILTER_DATE_FROM = None  # e.g., "20251001"
EMPLOYEE_FILTER_DATE_TO = None    # e.g., "20251031"

# Build WHERE clause dynamically
def get_employee_filter_clause():
    conditions = []
    if EMPLOYEE_FILTER_WEBSITE:
        conditions.append(f"LOWER(websitename) LIKE '%{EMPLOYEE_FILTER_WEBSITE.lower()}%'")
    if EMPLOYEE_FILTER_DATE_FROM:
        conditions.append(f"datekey >= {EMPLOYEE_FILTER_DATE_FROM}")
    if EMPLOYEE_FILTER_DATE_TO:
        conditions.append(f"datekey <= {EMPLOYEE_FILTER_DATE_TO}")
    return "WHERE " + " AND ".join(conditions) if conditions else ""

print("Employee Analysis Filters:")
print(f"  Website: {EMPLOYEE_FILTER_WEBSITE or 'All'}")
print(f"  Date from: {EMPLOYEE_FILTER_DATE_FROM or 'All'}")
print(f"  Date to: {EMPLOYEE_FILTER_DATE_TO or 'All'}")

In [ ]:
# Check if employee data is available
has_employee = 'fact_daily_employee' in all_db_tables

if has_employee:
    filter_clause = get_employee_filter_clause()
    
    # Metrics by business division
    division_summary = con.execute(f"""
        SELECT
            employeebusinessdivision,
            SUM(unique_visitors) as total_uv_sum,
            SUM(views) as total_views,
            SUM(visits) as total_visits,
            SUM(likes) as total_likes,
            SUM(comments) as total_comments,
            SUM(engagements) as total_engagements
        FROM fact_daily_employee
        {filter_clause}
        GROUP BY employeebusinessdivision
        ORDER BY total_views DESC
    """).df()
    
    print("Metrics by Business Division:")
    display(format_df(division_summary))
else:
    print("Employee data not available. Run ingest with employee_contact.csv to enable.")

In [ ]:
if has_employee:
    # Metrics by region
    region_summary = con.execute(f"""
        SELECT
            employeeregion,
            SUM(unique_visitors) as total_uv_sum,
            SUM(views) as total_views,
            SUM(engagements) as total_engagements,
            COUNT(DISTINCT date) as days_active
        FROM fact_daily_employee
        {filter_clause}
        GROUP BY employeeregion
        ORDER BY total_views DESC
    """).df()
    
    print("Metrics by Employee Region:")
    display(format_df(region_summary))

In [ ]:
if has_employee:
    # Cross-dimensional: Division by Website
    division_website = con.execute(f"""
        SELECT
            websitename,
            employeebusinessdivision,
            SUM(unique_visitors) as total_uv_sum,
            SUM(views) as total_views
        FROM fact_daily_employee
        {filter_clause}
        GROUP BY websitename, employeebusinessdivision
        ORDER BY total_views DESC
        LIMIT 25
    """).df()
    
    print("Top 25 Website + Division Combinations by Views:")
    display(format_df(division_website))

---
## 8. Visualizations

In [ ]:
# Daily UV trend by website
daily_trend = con.execute("""
    SELECT
        date,
        SUM(unique_visitors) as total_uv,
        SUM(views) as total_views
    FROM fact_daily_website
    GROUP BY date
    ORDER BY date
""").df()

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(daily_trend['date'], daily_trend['total_uv'], color='steelblue', linewidth=1.5)
ax.fill_between(daily_trend['date'], daily_trend['total_uv'], alpha=0.3, color='steelblue')

# Add 7-day moving average
if len(daily_trend) >= 7:
    daily_trend['ma7'] = daily_trend['total_uv'].rolling(window=7).mean()
    ax.plot(daily_trend['date'], daily_trend['ma7'], color='darkblue', 
            linewidth=2, linestyle='--', label='7-day Moving Avg')

ax.set_xlabel('Date')
ax.set_ylabel('Unique Visitors (sum)')
ax.set_title('Daily Unique Visitors Trend (Aggregated)')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"Average daily UV: {daily_trend['total_uv'].mean():,.0f}")
print(f"Peak UV: {daily_trend['total_uv'].max():,}")

In [ ]:
# UV by website bar chart
website_uv = con.execute("""
    SELECT
        websitename,
        SUM(unique_visitors) as total_uv,
        SUM(views) as total_views
    FROM fact_daily_website
    GROUP BY websitename
    ORDER BY total_uv DESC
""").df()

fig, ax = plt.subplots(figsize=(12, max(6, len(website_uv) * 0.4)))
bars = ax.barh(website_uv['websitename'], website_uv['total_uv'], color='seagreen')
ax.set_xlabel('Unique Visitors (sum)')
ax.set_ylabel('Website')
ax.set_title('Total UV by Website')
ax.invert_yaxis()

for bar, val in zip(bars, website_uv['total_uv']):
    ax.text(val + max(website_uv['total_uv']) * 0.01, bar.get_y() + bar.get_height()/2, 
            f'{val:,}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Close connection when done
# con.close()